# Sendy Regression Challenge Predict

## Importing the libraries

In [86]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats

## Importing the dataset

In [87]:
Train = pd.read_csv('Train.csv')
Riders = pd.read_csv('Riders.csv')
Test = pd.read_csv('Test.csv')

In [88]:
#Merging the datasets
total=Train.merge(Riders, how='left', on='Rider Id')

In [89]:
#Creating New Column
total['Time from Pickup to Arrival New']= total['Time from Pickup to Arrival']

In [90]:
#Dropping unneeded columns(Vehihle Type, User Id)
total = total.drop(['Vehicle Type'], axis=1)
total = total.drop(['User Id'], axis=1)

In [91]:
total = total.drop(['Time from Pickup to Arrival'], axis=1)

In [92]:
#Renaming the dependent variable column back to Time from Pickup to Arrival
total.rename(columns={'Time from Pickup to Arrival New':'Time from Pickup to Arrival'},inplace=True)

In [93]:
total.head()

,Order No,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,...,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Time from Pickup to Arrival
0,Order_No_4211,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,...,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,1637,1309,13.8,549,745
1,Order_No_25375,3,Personal,12,5,11:16:16 AM,12,5,11:23:21 AM,12,...,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,396,339,13.6,69,1993
2,Order_No_1899,3,Business,30,2,12:39:25 PM,30,2,12:42:44 PM,30,...,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,1023,242,12.5,114,455
3,Order_No_9336,3,Business,15,5,9:25:34 AM,15,5,9:26:05 AM,15,...,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,886,283,14.5,113,1341
4,Order_No_27883,1,Personal,13,1,9:55:18 AM,13,1,9:56:18 AM,13,...,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,2311,872,14.1,533,1214


In [94]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21201 entries, 0 to 21200
Data columns (total 31 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Order No                                   21201 non-null  object 
 1   Platform Type                              21201 non-null  int64  
 2   Personal or Business                       21201 non-null  object 
 3   Placement - Day of Month                   21201 non-null  int64  
 4   Placement - Weekday (Mo = 1)               21201 non-null  int64  
 5   Placement - Time                           21201 non-null  object 
 6   Confirmation - Day of Month                21201 non-null  int64  
 7   Confirmation - Weekday (Mo = 1)            21201 non-null  int64  
 8   Confirmation - Time                        21201 non-null  object 
 9   Arrival at Pickup - Day of Month           21201 non-null  int64  
 10  Arrival at Pickup - We

In [95]:
X = total.iloc[:, :-1].values
y = total.iloc[:, -1].values

## Taking care of missing data

In [97]:
#Temperature - replace null values with mean of the columns
from sklearn.impute import SimpleImputer
imp_temp = SimpleImputer(missing_values = np.nan, strategy='mean')
imp_temp.fit(total.iloc[:, 18:20]) 
total.iloc[:, 18:20] = imp_temp.transform(total.iloc[:, 18:20])

In [98]:
#Precipitation(ml) - replace null values with 0
from sklearn.impute import SimpleImputer
imp_prec = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value = 0)
imp_prec.fit(total.iloc[:,20:]) 
total.iloc[:,20:] = imp_prec.transform(total.iloc[:,20:])

## Changing datatype for Time columns


In [99]:
total['Placement - Time'] = pd.to_datetime(total['Placement - Time'])

In [100]:
total['Confirmation - Time'] = pd.to_datetime(total['Confirmation - Time'])

In [101]:
total['Arrival at Pickup - Time'] = pd.to_datetime(total['Arrival at Pickup - Time'])

In [102]:
total['Pickup - Time'] = pd.to_datetime(total['Pickup - Time'])

In [103]:
total['Arrival at Destination - Time'] = pd.to_datetime(total['Arrival at Destination - Time'])

## Encoding categorical data

In [104]:
#Personal or Business Business=1 and Personal=0
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [2])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [105]:
X[0]

array([1.0, 0.0, 'Order_No_4211', 3, 9, 5, '9:35:46 AM', 9, 5,
       '9:40:10 AM', 9, 5, '10:04:47 AM', 9, 5, '10:27:30 AM', 9, 5,
       '10:39:55 AM', 4, 20.4, nan, -1.3177546999999998, 36.8303703,
       -1.3004062, 36.829741, 'Rider_Id_432', 1637, 1309, 13.8, 549],
      dtype=object)

## Splitting the dataset into the Training set and Test set

In [107]:
#Drop columns not wanted to be part of the model
total_ln = total.iloc[:, 12:]
total_ln.drop('Rider Id', axis=1, inplace=True)
total_ln.drop('Pickup Long', axis=1, inplace=True)
total_ln.drop('Pickup Lat', axis=1, inplace=True)
total_ln.drop('Arrival at Destination - Time', axis=1, inplace=True)
total_ln.drop('Pickup - Time', axis=1, inplace=True) 
total_ln.drop('Destination Lat', axis=1, inplace=True)
total_ln.drop('Destination Long', axis=1, inplace=True)
#total_ln.drop('No_Of_Orders', axis=1, inplace=True)
total_ln.drop('Age', axis=1, inplace=True)
#total_ln.drop('Average_Rating', axis=1, inplace=True)
total_ln.drop('No_of_Ratings', axis=1, inplace=True)


total_ln.info()
X = total_ln.iloc[:,:-1]
y = total_ln.iloc[:,-1]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21201 entries, 0 to 21200
Data columns (total 10 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Pickup - Day of Month                      21201 non-null  int64  
 1   Pickup - Weekday (Mo = 1)                  21201 non-null  int64  
 2   Arrival at Destination - Day of Month      21201 non-null  int64  
 3   Arrival at Destination - Weekday (Mo = 1)  21201 non-null  int64  
 4   Distance (KM)                              21201 non-null  float64
 5   Temperature                                21201 non-null  float64
 6   Precipitation in millimeters               21201 non-null  float64
 7   No_Of_Orders                               21201 non-null  int64  
 8   Average_Rating                             21201 non-null  float64
 9   Time from Pickup to Arrival                21201 non-null  int64  
dtypes: float64(4), int64(6

In [108]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Training the Multiple Linear Regression model on the Training set

In [109]:
# Multiple Linear regression model 
from sklearn.linear_model import LinearRegression
mlr = LinearRegression()
mlr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [110]:
#Decision tree regression model 
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)


DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [111]:
#Random forest regression model 
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators = 25)
forest.fit(X_test, y_test)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=25, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

## Predicting the Test set results

In [112]:
from sklearn import metrics
import math

rmse_test = round(math.sqrt(metrics.mean_squared_error(y_test, mlr.predict(X_test))),2)
rmse_train = round(math.sqrt(metrics.mean_squared_error(y_train, mlr.predict(X_train))),2)

[rmse_test, rmse_train]

[798.32, 801.09]

In [113]:
# Decision Tree regression
from sklearn import metrics
import math

rmse_test1 = round(math.sqrt(metrics.mean_squared_error(y_test, tree.predict(X_test))),2)
rmse_train1 = round(math.sqrt(metrics.mean_squared_error(y_train, tree.predict(X_train))),2)

[rmse_test1, rmse_train1]

[1119.0, 55.62]

In [114]:
# Random forrest regression 

rmse_test2 = round(math.sqrt(metrics.mean_squared_error(y_test, forest.predict(X_test))),2)
        
rmse_train2 = round(math.sqrt(metrics.mean_squared_error(y_train, forest.predict(X_train))),2)

[rmse_test2, rmse_train2]
                    

[320.98, 830.33]

In [116]:
data = {'Regression model':  ['MLR', 'D.Tree', 'R.Forest'],
        'RMSE Training': [rmse_train, rmse_train1, rmse_train2],
        'RMSE Testing' : [rmse_test, rmse_test1, rmse_test2]
        }

df = pd.DataFrame (data, columns = ['Regression model', 'RMSE Training', 'RMSE Testing'])
print(df)

  Regression model  RMSE Training  RMSE Testing
0              MLR         801.09        798.32
1           D.Tree          55.62       1119.00
2         R.Forest         830.33        320.98


## MLR coefficients


In [76]:
beta_0 = float(mlr.intercept_)

In [77]:
beta_js = pd.DataFrame(mlr.coef_, X.columns, columns=['Coefficient'])

In [78]:
print("Intercept:", beta_0)

Intercept: 1077.837507479573


In [79]:
beta_js

,Coefficient
Pickup - Day of Month,-0.793672
Pickup - Weekday (Mo = 1),1.421320
Arrival at Destination - Day of Month,-0.793672
Arrival at Destination - Weekday (Mo = 1),1.421320
Distance (KM),100.642723
Temperature,5.334063
Precipitation in millimeters,2.650221
No_Of_Orders,-0.102678
Age,0.009802
Average_Rating,-37.254328


In [80]:
# dictionary of results
results_dict = {'Training MSE':
                    {
                        "MLR": metrics.mean_squared_error(y_train, mlr.predict(X_train))
                    },
                'Test MSE':
                    {
                        "MLR": metrics.mean_squared_error(y_test, mlr.predict(X_test))
                    },
                'Test RMSE':
                    {
                        "MLR": math.sqrt(metrics.mean_squared_error(y_test, mlr.predict(X_test)))
                    }
                }

In [81]:
results_df = pd.DataFrame(data=(results_dict))
results_df

,Training MSE,Test MSE,Test RMSE
MLR,638298.894941,633157.584306,795.711998


In [82]:
testing = Test.merge(Riders, how='left', on='Rider Id')

#Test Data
testing = testing.drop(['Vehicle Type'], axis=1)
testing = testing.drop(['User Id'], axis=1)

X1 = testing.iloc[:, :].values

#Distance/Temperature
from sklearn.impute import SimpleImputer
imp_temp = SimpleImputer(missing_values = np.nan, strategy='mean')
imp_temp.fit(testing.iloc[:, 15:17]) 
testing.iloc[:, 15:17] = imp_temp.transform(testing.iloc[:, 15:17])

#Precipitation(ml)
from sklearn.impute import SimpleImputer
imp_prec = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value = 0)
imp_prec.fit(testing.iloc[:,17:]) 
testing.iloc[:,17:] = imp_prec.transform(testing.iloc[:,17:])

testing['Placement - Time'] = pd.to_datetime(testing['Placement - Time'])
testing['Confirmation - Time'] = pd.to_datetime(testing['Confirmation - Time'])
testing['Arrival at Pickup - Time'] = pd.to_datetime(testing['Arrival at Pickup - Time'])
testing['Pickup - Time'] = pd.to_datetime(testing['Pickup - Time'])

#Personal or Business
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [2])], remainder='passthrough')
X1 = np.array(ct.fit_transform(X1))

In [83]:
X1

array([[1.0, 0.0, 'Order_No_19248', ..., 1511, 13.3, 171],
       [1.0, 0.0, 'Order_No_12736', ..., 273, 14.4, 45],
       [1.0, 0.0, 'Order_No_768', ..., 460, 13.1, 67],
       ...,
       [1.0, 0.0, 'Order_No_1969', ..., 1337, 13.7, 441],
       [0.0, 1.0, 'Order_No_10591', ..., 878, 14.1, 66],
       [1.0, 0.0, 'Order_No_1603', ..., 297, 14.0, 108]], dtype=object)

In [84]:
testing.head(2)

,Order No,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,...,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Order_No_19248,3,Business,27,3,2020-05-18 16:44:10,27,3,2020-05-18 16:44:29,27,...,0.0,-1.333275,36.870815,-1.305249,36.822390,Rider_Id_192,439,1511,13.3,171
1,Order_No_12736,3,Business,17,5,2020-05-18 12:57:35,17,5,2020-05-18 12:59:17,17,...,0.0,-1.272639,36.794723,-1.277007,36.823907,Rider_Id_868,488,273,14.4,45


In [85]:
#TESTING MODEL ON TEST DATA(CSV)

#factors that will predict Time from Pickup to Arrival
desired_factors = ['Distance (KM)','Temperature','Precipitation in millimeters'
                   ,'No_Of_Orders','Age', 'Average_Rating','No_of_Ratings']

#set my model to DecisionTree
model = LinearRegression()
#model = RandomForestRegressor
#model = DecisionTreeRegressor

#set prediction data to factors that will predict, and set target to (time Arrival from Pick up to Arrival)
train_data = total[desired_factors]
test_data = testing[desired_factors]
target = total['Time from Pickup to Arrival']

#fitting model with prediction data and telling it my target
model.fit(train_data, target)

results = model.predict(test_data)
df = pd.DataFrame (results, columns = ['Time from Pickup to Arrival'])
df.head()

,Time from Pickup to Arrival
0,1513.867433
1,1114.251836
2,1137.056454
3,1149.909348
4,1071.405675
